# Benchmark simulations on an idealised alveolar mesh

The following notebook contains sample code for simulating gas exchange on an idealised alveolar geometry using the model outlined in Zurita & Hurtado (2022).

First we import necessary libraries and add important directories to the system path.

In [1]:
import sys
import os
import dolfin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
sys.path.append(os.getcwd()[:-6])

Following that, we import the important modules from the model source files.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Now we outline the folder name and the path to save raw data files.

In [3]:
folder = "sphere_refined_small"
path = os.path.join("../raw-data", folder)

In [4]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.import_mesh(os.path.join("../raw-data", "sphere_coarse_small.xml"), type="xml")
model.sim_p(save=True, meshtype="sphere")
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries
x = model.sim_sbst(hb=False, save=False)
solution = model.sim_sbst(hb=True, save=True, guess=x)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.741e+08 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-08)
  Newton iteration 1: r (abs) = 1.931e-07 (tol = 1.000e-08) r (rel) = 7.045e-16 (tol = 1.000e-08)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.563e+10 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-08)
  Newton iteration 1: r (abs) = 2.645e+08 (tol = 1.000e-08) r (rel) = 1.692e-02 (tol = 1.000e-08)
  Newton iteration 2: r (abs) = 1.658e+07 (tol = 1.000e-08) r (rel) = 1.061e-03 (tol = 1.000e-08)
  Newton iterat